In [1]:
import json
from my_utils import NERDataset,collate_fn
from torch.utils.data import Dataset,DataLoader

In [5]:
dataset = NERDataset('nlp2024-data/dataset/dev.json')
dataloader = DataLoader(dataset,batch_size=8,collate_fn=collate_fn)
for batch in dataloader:
    print(batch)
    break

(['你好，咳嗽是连声咳吗？有痰吗？有没流鼻涕，鼻塞？', '咳嗽有几天了？', '有发热过吗？', '有三天', '没发烧，也没痰鼻塞', '以前有气喘吗？', '有没什么过敏？', '没有'], [[10, 10, 10, 4, 9, 10, 10, 10, 4, 10, 10, 10, 4, 10, 10, 10, 10, 4, 9, 9, 10, 4, 9, 10], [4, 9, 10, 10, 10, 10, 10], [10, 4, 9, 10, 10, 10], [10, 10, 10], [10, 4, 9, 10, 10, 10, 4, 4, 9], [10, 10, 10, 4, 9, 10, 10], [10, 10, 10, 10, 4, 9, 10], [10, 10]])


(['大便怎么样？干不干？胃口怎么样？', "大便经常干'胃口很好", '可能有点积食', '那该总么办', '磨牙，晚上翻来覆去，大便干，吃的多，很容易积食', '现在可以吃点小儿消积止咳口服液', '如果没有这个，可以吃点健儿清解液，小儿消食颗粒', '益生菌也可以吃点。'], [[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [10, 10, 10, 10, 4, 9], [10, 10, 10, 10, 10], [4, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 4, 9], [10, 10, 10, 10, 10, 10, 0, 5, 5, 5, 5, 5, 5, 5, 5], [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 5, 5, 5, 5, 10, 0, 5, 5, 5, 5, 5], [1, 6, 6, 10, 10, 10, 10, 10, 10]])
